# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
output_data_file = "output_data/cities.csv"

output_data = pd.read_csv(output_data_file)

output_data.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
gmaps.configure(api_key=g_key)

locations = output_data[['Lat', 'Lng']]
weight = output_data['Humidity']

fig = gmaps.figure(center=(15.34, 74.49), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weight))
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
weather_conditions=output_data.loc[(output_data['Max Temp']>70)&(output_data['Max Temp']<80)&(output_data['Cloudiness']==0)&(output_data['Wind Speed']<10)]
weather_conditions1 = weather_conditions.dropna()
weather_conditions1.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


In [5]:
weather_conditions1

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Set up hotel DF

hotel_df = weather_conditions1
#cleaned up below
    #hotel_name = ""
    #hotel_df["Hotel Name"] = hotel_name
hotel_df["Hotel Name"] = ""
hotel_df


<ipython-input-6-2be2c8bc6832>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,


In [7]:
# Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# Set up a parameters dictionary
params = {"keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key}



In [8]:
# Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# Set up a parameters dictionary
params = {"keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key,
         "location": "32.86,59.22"}

# test before larger search
response = requests.get(base_url, params=params).json()

results = response['results']
results

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 32.8802023, 'lng': 59.216279},
   'viewport': {'northeast': {'lat': 32.88151937989272,
     'lng': 59.21771057989272},
    'southwest': {'lat': 32.87881972010727, 'lng': 59.21501092010728}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'icon_background_color': '#909CE1',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
  'name': 'هتل پرويز',
  'place_id': 'ChIJ8dNk011nGj8RZI51wiSCBsY',
  'rating': 3,
  'reference': 'ChIJ8dNk011nGj8RZI51wiSCBsY',
  'scope': 'GOOGLE',
  'types': ['lodging', 'point_of_interest', 'establishment'],
  'user_ratings_total': 1,
  'vicinity': 'V6J8+3GJ, Birjand'},
 {'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 32.8851325, 'lng': 59.2167535},
   'viewport': {'northeast': {'lat': 32.88649142989272,
     'lng': 59.21818162989271},
    'southwest': {'lat': 32.88379177010728, 'lng': 5

In [9]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    city_name = row['City']
    lat_loc = row['Lat']
    lng_loc = row['Lng'] 

    
    # add location to params dict
    params['location'] = f'{lat_loc},{lng_loc}'


# # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}")
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    try:
        print(f" In {city_name} the closest hotel is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
          
    except (KeyError, IndexError):
        print("No hotel ... skipping")
        

Retrieving Results for Index 70: ponta do sol
 In ponta do sol the closest hotel is Escarpas Resort
Retrieving Results for Index 88: cidreira


/Users/sreiss/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


 In cidreira the closest hotel is Hotel Castelo
Retrieving Results for Index 138: morondava
 In morondava the closest hotel is Palissandre Cote Ouest resort & SPA
Retrieving Results for Index 139: vaini
No hotel ... skipping
Retrieving Results for Index 176: nador
 In nador the closest hotel is Hôtel Mercure Rif Nador
Retrieving Results for Index 297: mogok
 In mogok the closest hotel is King Bridge
Retrieving Results for Index 444: birjand
 In birjand the closest hotel is هتل پرويز


In [10]:
hotel_df

#should I create a new df to null the missing values?

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Escarpas Resort
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Mercure Rif Nador
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,King Bridge
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,هتل پرويز


In [29]:
# Add marker layer ontop of heat map

gmaps.configure(api_key=g_key)

marker_locations = hotel_df[['Lat','Lng']]

# marker_locations = [(-20.63,-46),(-30.17,-50.22),(-20.30, 44), (35.17, -2.93), (22.92, 96.51), (32.86, 59.22)]
#fig = gmaps.figure()
fig = gmaps.figure(center=(15.34, 74.49), zoom_level=2)
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))

In [30]:
# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

gmaps.configure(api_key=g_key)

locations = output_data[['Lat', 'Lng']]
weight = output_data['Humidity']
marker_locations = hotel_df[['Lat','Lng']]
# marker_locations = [(-20.63,-46),(-30.17,-50.22),(-20.30, 44), (35.17, -2.93), (22.92, 96.51), (32.86, 59.22)]
markers = gmaps.marker_layer(marker_locations)

#fig = gmaps.figure()
fig = gmaps.figure(center=(15.34, 74.49), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weight))
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

In [33]:
#adding to markers to the tags - working off examples I found online below

#  Don't need these below, exporting from the dataframe
# marker_loc = hotel_df[['Lat','Lng']]
# marker_city = hotel_df[["City"]]
# marker_country = hotel_df[["Country"]]
# marker_hotel = hotel_df[["Hotel Name"]]


info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

marker_loc = hotel_df[['Lat','Lng']]
hotel_info = [info_box_template.format(**row)for index, row in hotel_df.iterrows()]
#fig = gmaps.figure()
fig = gmaps.figure(center=(15.34, 74.49), zoom_level=2)
marker_layer = gmaps.marker_layer(marker_loc, info_box_content=hotel_info)

fig.add_layer(gmaps.heatmap_layer(locations, weights=weight))
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))